In [18]:
'''

Functions for obtaining the morphological features.  Goals:

• Derivational
- hand-compiled list by Hancke et al (2012) 

• Inflectional
- mood, person, tense, type of verbs
- case of nouns

• Compound words 
- Jwordsplitter (http://www.danielnaber.de/jwordsplitter/)

Notes:
- finish derivational Hancke features, JWordSplitter
- implement on actual dataset (expect new formatting problems)
- add classifications

next-- lexical features (Hancke thesis)


'''
from nltk import word_tokenize
import numpy as np
import pandas as pd
import urllib.request
import io
import ast

# first we need to pre-process the text by identifying each sentence and splitting it into one token per line for
# the RFTagger.

url = "https://raw.githubusercontent.com/soerenetler/TextDifficultyAssessmentGerman/master/preprocessed_text_df.csv"
df = pd.read_csv(url, encoding='utf8', engine='python')
df = df.drop(['Unnamed: 0'],axis=1) # drop extra index column added in CSV file
display(df.head())

'''
document = open("test.txt","r")
file = open("testtok.txt","w")
sentences = []

for line in document:
    index = line.index(".")
    sentences.append(line[:index]) # keep period, but cut off newline \n which appears

for sentence in sentences:
    tokens = word_tokenize(sentence)
    for token in tokens:
        file.write(token)
        file.write("\n")
    file.write(".")
    file.write("\n")
    file.write("\n")
            
file.close()

# use the RFTagger, which takes a text file of word-tokenized sentences (one token per line) and annotates each one like so:
# 0Das\tPRO.Dem.Subst.-.Nom.Sg.Neut

file = ! RFTagger/src/rft-annotate RFTagger/lib/german.par testtok.txt
file = file[2:-3]

# takes a file as input and returns all annotated sentences (each word is annotated)
def get_annotated_sentences(file):

    annotated_sentences = [] # this will collect each annotated sentence as a list

    second_newline = False # due to RFTagger formatting, there are 2 newlines before each new sentence
    sentence = [] # each annotated token is added to this during a sentence, then added to annotated_sentences after

    for line in file:
        if not line and not second_newline:
            second_newline = True
            annotated_sentences.append(sentence)
            sentence = []
        
        if line:
            sentence.append(line)
            second_newline = False
    
    annotated_sentences.append(sentence) # append the last sentence as well since this doesn't get added in loop
    return annotated_sentences

# a helper function to split each annotation into its components-- a word and its morphological features
def splitter(tokens):
    word,features = tokens.split("\t")
    list_of_features = features.split(".")
    return word, list_of_features
'''
# returns a list of all the features included in the dataset, to be used as attributes in feature vector
def get_all_features(df):
    all_features = [] 
    sentence_count = 0

    for all_sentences in df.loc[:,'preprocessedText']:
        if all_sentences is not None:
            all_sentences = all_sentences[:-5] # remove the empty last sentence [] as following line won't work
            all_sentences += "]" # nasty preprocessing necessary to keep list structure for literal_eval
            all_sentences = ast.literal_eval(all_sentences)
            for sentence in all_sentences:
                sentence_count +=1
                for word_features in sentence:
                    word_features = word_features[1:-1] #exclude the word itself and lemma at end (leads to too many features)
                    for token in word_features:
                        if token == "APPO":
                            print ("HERE IT IS")
                        if token not in all_features:
                            all_features.append(token)
        else: break # catch empty all_sentences case
                    
    return sentence_count, all_features

def get_classifications(df):
    y = []
    
    for classification in df.loc[:,'Level']:
        y.append(classification)
        
    return y

def get_classifications_sentences(df):
    return

def fill_sentence_vector(df,df_sentence):
    return

def fill_document_vector(df,df_document):
    index = 0
    print(list(df_document))
    for all_sentences in df.loc[:,'preprocessedText']:
        if all_sentences is not None:
            if index == 414:
                print(all_sentences)
            all_sentences = all_sentences[:-5] # remove the empty last sentence [] as following line won't work
            all_sentences += "]" # nasty preprocessing necessary to keep list structure for literal_eval
            all_sentences = ast.literal_eval(all_sentences)
            for sentence in all_sentences:
                for word_features in sentence:
                    word_features = word_features[1:-1] #exclude the word itself and lemma at end (leads to too many features)
                    for token in word_features:
                        #print(token)
                        print(index)
                        df_document.loc[index,token] +=1
            index +=1


# get total number of sentences and all features contained in dataset in order to create vector columns

num_sentences, all_features = get_all_features(df)  

'''
# sentence level vector, i.e. each row in the feature vector represents one sentence

zero_data = np.zeros(shape=(num_sentences,len(all_features)))
df_sentence = pd.DataFrame(zero_data, columns=sorted(all_features))

y = get_classifications(df)

fill_sentence_vector(df,df_sentence)
'''

# now document level vector, i.e. each row in the feature vector represents one document

zero_data = np.zeros(shape=(len(df),len(all_features))) # create 1 row in the feature vector per document
df_document = pd.DataFrame(zero_data,columns=sorted(all_features))
y = get_classifications(df)

fill_document_vector(df,df_document)
display(df_document.head())


'''
having trouble getting the JWordSplitter to work... don't find the .jar file and can't seem to make it from the
provided files

! java -jar jwordsplitter-x.y.jar tokgerman.txt
'''

,Level,Title,Text,Source,Type,preprocessedText,parsedText
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading,"[[['Zu', 'APPR', 'zu'], ['meiner', 'PRO', 'Pos...",['(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN F...
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading,"[[['Mein', 'PRO', 'Poss', 'Attr', 'Nom', 'Sg',...",['(ROOT (S (NP (PPOSAT Mein) (NN Name)) (VAFIN...
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading,"[[['Hallo', 'ITJ', 'hallo'], ['!', 'SYM', 'Pun...","['(ROOT (NUR (ITJ Hallo) ($. !)))', '(ROOT (S ..."
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading,"[[['Ricarda', 'N', 'Name', 'Nom', 'Sg', 'Fem',...",['(ROOT (CS (S (NE Ricarda) (VAFIN ist) (AP (N...
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading,"[[['Frau', 'N', 'Reg', 'Nom', 'Sg', 'Fem', 'Fr...",['(ROOT (S (NP (NN Frau) (NE Meier)) (VVFIN ge...


['*', '1', '2', '3', 'ADJA', 'ADJD', 'ADV', 'APPR', 'APPRART', 'APZR', 'ART', 'Acc', 'Ans', 'Attr', 'Aux', 'CARD', 'CONJ', 'Colon', 'Comma', 'Comp', 'Cont', 'Coord', 'Dat', 'Def', 'Deg', 'Dem', 'FM', 'Fem', 'Full', 'Gen', 'Hyph', 'ITJ', 'Ind', 'Indef', 'Inter', 'Left', 'Masc', 'Mod', 'N', 'Name', 'Neg', 'Neut', 'Nom', 'Noun', 'Other', 'PART', 'PRO', 'PROADV', 'Paren', 'Past', 'Pers', 'Pl', 'Pos', 'Poss', 'Pres', 'Psp', 'Pun', 'Quot', 'Refl', 'Reg', 'Rel', 'Right', 'SYM', 'Sent', 'Sg', 'Slash', 'SubFin', 'SubInf', 'Subj', 'Subst', 'Sup', 'TRUNC', 'VFIN', 'VIMP', 'VINF', 'VPP', 'Verb', 'XY', 'Zu', 'zu']
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7


13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
1

18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
1

...

373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374


385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386
386


397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
397
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398
398


408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409
409


KeyError: 'the label [APPO] is not in the [index]'